<h1 align="center"> Sensordatenfusion Tutorium 02 </h1>

<h3 align="center"> Erstellen des Transitionsmodells </h3>

In [ ]:
import scipy as sp
from stonesoup import *
from scipy.stats import multivariate_normal
from scipy.linalg import block_diag

from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import (LinearModel, GaussianModel, TimeVariantModel,
                    TimeInvariantModel)
from stonesoup.base import Property
from stonesoup.models.transition import TransitionModel
from stonesoup.models.transition.linear import LinearGaussianTransitionModel

In [ ]:
class PCWAModel(LinearGaussianTransitionModel, TimeVariantModel):
    
    def matrix(self, time_interval, **kwargs):
        return sp.array([[1, time_interval.total_seconds()], [0, 1]])
    
    
    def covar(self, time_interval, **kwargs):
        time_interval_sec = time_interval.total_seconds()
        
        Sigma = 5
        
        covar = sp.array([[sp.power(time_interval_sec, 3)/3,
                           sp.power(time_interval_sec, 2)/2],
                          [sp.power(time_interval_sec, 2)/2,
                           time_interval_sec]]) * Sigma
        
        return CovarianceMatrix(covar)
